<a href="https://colab.research.google.com/github/Arwa008/Cybersecurity/blob/main/FS_Network_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [216]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import regularizers
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')
%matplotlib inline

In [217]:
# Read Train and Test dataset
data = pd.read_csv("KDDTrain.txt")
data.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.18,2,2.1,0.00,0.00.1,0.00.2,0.00.3,1.00,0.00.4,0.00.5,150,25,0.17,0.03,0.17.1,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [218]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations'
,'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate'
,'srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','outcome','level'])
data.columns = columns
display(data.head())
print(data.shape)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,level
0,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


(125972, 43)


In [219]:
# put labels
data.loc[data['outcome'] == "normal", "outcome"] = 'normal'
data.loc[data['outcome'] != 'normal', "outcome"] = 'attack'

# Preprocessing

## 1- Split and Encode data

In [220]:
from sklearn.preprocessing import LabelEncoder

# Split to data and target
X = data.drop("outcome", axis=1)
y = data["outcome"]

# Enode Target
le = LabelEncoder()
y = le.fit_transform(y)

# Check categorical features
Cate_Features=X.select_dtypes(include=['object']).columns.tolist()
print(Cate_Features)

# Enode Cate_Features
enc_protocol = pd.get_dummies(data["protocol_type"], prefix="protocol_")
enc_service = pd.get_dummies(data["service"], prefix="service_")
enc_flag = pd.get_dummies(data["flag"], prefix="flag_")
X = pd.concat([X, enc_protocol, enc_service, enc_flag], axis=1)
X.drop("protocol_type", axis=1, inplace=True)
X.drop("service", axis=1, inplace=True)
X.drop("flag", axis=1, inplace=True)

#Display encoded dataframes
y1=pd.DataFrame(y)
y1.columns=['outcome']
display(X.head())
print(X.shape)
display(y1.head())
print(y1.shape)


['protocol_type', 'service', 'flag']


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,protocol__icmp,protocol__tcp,protocol__udp,service__IRC,service__X11,service__Z39_50,service__aol,service__auth,service__bgp,service__courier,service__csnet_ns,service__ctf,service__daytime,service__discard,service__domain,service__domain_u,service__echo,service__eco_i,service__ecr_i,service__efs,service__exec,service__finger,service__ftp,service__ftp_data,service__gopher,service__harvest,service__hostnames,service__http,service__http_2784,service__http_443,service__http_8001,service__imap4,service__iso_tsap,service__klogin,service__kshell,service__ldap,service__link,service__login,service__mtp,service__name,service__netbios_dgm,service__netbios_ns,service__netbios_ssn,service__netstat,service__nnsp,service__nntp,service__ntp_u,service__other,service__pm_dump,service__pop_2,service__pop_3,service__printer,service__private,service__red_i,service__remote_job,service__rje,service__shell,service__smtp,service__sql_net,service__ssh,service__sunrpc,service__supdup,service__systat,service__telnet,service__tftp_u,service__tim_i,service__time,service__urh_i,service__urp_i,service__uucp,service__uucp_path,service__vmnet,service__whois,flag__OTH,flag__REJ,flag__RSTO,flag__RSTOS0,flag__RSTR,flag__S0,flag__S1,flag__S2,flag__S3,flag__SF,flag__SH
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,15,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,19,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


(125972, 123)


,outcome
0,1
1,0
2,1
3,1
4,0


(125972, 1)


##2- Split data to train and test and Scaling

In [221]:
from sklearn import preprocessing
scaler =preprocessing.MinMaxScaler() #preprocessing.StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [222]:
# display(X_train_scaled.head())
# display(X_test_scaled.head())
# print('Train Shape: ',X_train_scaled.shape)
# print('Test Shape: ',X_test_scaled.shape)

#Feature selection

### 1-Princaple Component Analysis

In [223]:
pca = PCA(n_components=20)

x_train1=X_train_scaled.copy()
x_test1=X_test_scaled.copy()

Train_pca = pca.fit_transform(x_train1)
Test_pca = pca.transform(x_test1)

### 2-Chi square

In [224]:
x_train2=X_train_scaled.copy()
x_test2=X_test_scaled.copy()
#chi2Score=chi2(x_train2,y_train)
K_best_features = SelectKBest(score_func=chi2,k=20)

Train_chi = K_best_features.fit_transform(x_train2,y_train)
Test_chi = K_best_features.transform(x_test2)

### 3-Anova

In [229]:
x_train3=X_train_scaled.copy()
x_test3=X_test_scaled.copy()
from sklearn.feature_selection import f_classif

K_best_features1 = SelectKBest(f_classif, k=20)
Train_anova = K_best_features1.fit_transform(x_train3,y_train)
Test_anova = K_best_features1.transform(x_test3)

In [230]:
import time
resultsTrain = pd.DataFrame()
resultsTest = pd.DataFrame()
lr = LogisticRegression()
def evaluate_classification(model, X_train, X_test, y_train, y_test,YN):
    model.fit(X_train, y_train)

    t0=time.time()
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    tF = time.time()
    print('Time to train = %.2f seconds' % (tF - t0))

    shape=len(X_train[0])
    train_accuracy = '{:.2%}'.format(metrics.accuracy_score(y_train,y_train_pred))
    test_accuracy = '{:.2%}'.format(metrics.accuracy_score(y_test,y_test_pred))

    train_precision = '{:.2%}'.format(metrics.precision_score(y_train,y_train_pred))
    test_precision = '{:.2%}'.format(metrics.precision_score(y_test,y_test_pred))

    train_recall = '{:.2%}'.format(metrics.recall_score(y_train,y_train_pred))
    test_recall = '{:.2%}'.format(metrics.recall_score(y_test,y_test_pred))

    results1=resultsTrain.append({'Model':model,'F_S':YN,'No.features':shape,'Result':'Train','precision':train_precision,'recall':train_recall,'Accuracy':train_accuracy},ignore_index=True)
    results2=resultsTest.append({'Model':model,'F_S':YN,'No.features':shape,'Result':'Test','precision':test_precision,'recall':test_recall,'Accuracy':test_accuracy},ignore_index=True)
    frames = [results1, results2]
    Overall_result = pd.concat(frames)
    display (Overall_result)


## Modeling

### Logistic Regression

In [231]:
evaluate_classification(lr, X_train_scaled, X_test_scaled, y_train, y_test,'None')

Time to train = 0.07 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,LogisticRegression(),None,123,Train,99.21%,98.81%,98.94%
0,LogisticRegression(),None,123,Test,99.16%,98.64%,98.83%


In [232]:
evaluate_classification(lr, Train_pca, Test_pca, y_train, y_test,'PCA')

Time to train = 0.04 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,LogisticRegression(),PCA,20,Train,96.34%,97.73%,96.80%
0,LogisticRegression(),PCA,20,Test,96.36%,97.63%,96.76%


In [233]:
evaluate_classification(lr, Train_chi, Test_chi, y_train, y_test,'Chi2')

Time to train = 0.03 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,LogisticRegression(),Chi2,20,Train,95.52%,96.37%,95.65%
0,LogisticRegression(),Chi2,20,Test,95.60%,95.96%,95.47%


In [234]:
evaluate_classification(lr, Train_anova, Test_anova, y_train, y_test,'Anova')

Time to train = 0.01 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,LogisticRegression(),Anova,20,Train,98.29%,96.90%,97.44%
0,LogisticRegression(),Anova,20,Test,98.32%,96.77%,97.38%


### GaussianNB

In [235]:
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB(priors=None, var_smoothing=1e-09)

In [236]:
evaluate_classification(NB, X_train_scaled, X_test_scaled, y_train, y_test,'None')

Time to train = 0.25 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,GaussianNB(),None,123,Train,78.89%,99.78%,85.62%
0,GaussianNB(),None,123,Test,79.15%,99.77%,85.80%


In [237]:
evaluate_classification(NB, Train_pca, Test_pca, y_train, y_test,'PCA')

Time to train = 0.06 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,GaussianNB(),PCA,20,Train,88.27%,93.20%,89.75%
0,GaussianNB(),PCA,20,Test,88.14%,93.30%,89.69%


In [238]:
evaluate_classification(NB, Train_chi, Test_chi, y_train, y_test,'Chi')

Time to train = 0.03 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,GaussianNB(),Chi,20,Train,89.17%,92.27%,89.88%
0,GaussianNB(),Chi,20,Test,89.08%,92.03%,89.68%


In [239]:
evaluate_classification(NB, Train_anova, Test_anova, y_train, y_test,'Anova')

Time to train = 0.04 seconds


,Model,F_S,No.features,Result,precision,recall,Accuracy
0,GaussianNB(),Anova,20,Train,89.90%,91.70%,90.06%
0,GaussianNB(),Anova,20,Test,89.66%,91.62%,89.85%
